In [3]:
using POMDPs, POMDPModels, POMDPSimulators, ARDESPOT

using POMDPs
using Distributions: Normal
using Random
import POMDPs: initialstate_distribution, actions, gen, discount, isterminal
Random.seed!(1);

┌ Info: Precompiling POMDPSimulators [e0d0a172-29c6-5d4e-96d0-f262df5d01fd]
└ @ Base loading.jl:1273
┌ Info: Precompiling ARDESPOT [d96c9ae4-3372-47d5-8a88-316ae77be8cf]
└ @ Base loading.jl:1273


In [ ]:
pomdp = TigerPOMDP()

solver = DESPOTSolver(bounds=(-20.0, 0.0))
planner = solve(solver, pomdp)

for (s, a, o) in stepthrough(pomdp, planner, "s,a,o", max_steps=10)
    println("State was $s,")
    println("action $a was taken,")
    println("and observation $o was received.\n")
end

In [ ]:
actions(pomdp)

In [ ]:
using StructArrays

In [ ]:
struct human_goal_location
    x:: Int64
    y:: Int64
end

In [ ]:
s = StructArray{human_goal_location}

In [ ]:
h1 = human_goal_location(1,1) 
h2 = human_goal_location(2,2)

In [ ]:
# push!(s, h1)

In [ ]:
function lala(obj)
    @show obj.x
    @show obj.y
    h = human_goal_location(obj.x+5,obj.y-1)
    return h
end

In [ ]:
lala(h2)

In [ ]:
struct human_goal_location
    x:: Int64
    y:: Int64
end
struct pedestrian_state
    x:: Int64
    y:: Int64
    goal:: human_goal_location
end
struct cart_state
    x:: Int64
    y:: Int64
    theta:: Int64
    v:: Int64
end    
struct observations
    observed_human_positions:: Array{human_goal_location}
end
struct SP_POMDP_state
    cart:: cart_state
    pedestrians:: Array{pedestrian_state}
    pedestrian_goals:: Array{human_goal_location}
    path:: Array{Int64}
    path_covered_index:: Int64
end

In [ ]:
mutable struct Speed_Planner_POMDP <: POMDPs.POMDP{SP_POMDP_state, Int, observations}
    discount_factor::Float64
    step_size::Int
end

In [ ]:
# mutable struct LightDark1D <: POMDPs.POMDP{Float64,Int,Int}
#     discount_factor::Float64
#     correct_r::Float64
#     incorrect_r::Float64
#     #step_size::Int
#     movement_cost::Float64
# end
# LightDark1D() = LightDark1D(0.9, 10, -10, 1, 0)
# discount(p::LightDark1D) = p.discount_factor
# isterminal(::LightDark1D, s::Float64) = isnan(s);

In [ ]:
function POMDPs.gen(m::Speed_Planner_POMDP, s, a, rng)
    
    # transition model
    
    function calculate_theta(current_state, previous_state)
        theta = 0
        x_diff = current_state[1] - previous_state[1]
        y_diff = current_state[2] - previous_state[2]
        if x_diff != 0
            if x_diff < 0
                theta = 90
            else
                theta = 270
            end
        end
        if y_diff != 0
            if y_diff < 0
                theta = 0
            else
                theta = 180
            end
        end
        return theta
    end
    
    function update_human_state(human, human_goals, rng)
        goal = human.goal
        
        human_fixed_goals = copy(human_goals)
        deleteat!(human_fixed_goals, findall(x -> x==goal, human_fixed_goals)[1])
        
        rand_num = rand(rng)
        
        function move_human_towards_goal(human, goal)
            temp_human_x = human.x
            temp_human_y = human.y
            if temp_human_x < goal.x
                temp_human_x = temp_human_x + 1
            elseif temp_human_x > goal.x
                temp_human_x = temp_human_x - 1
            end

            if temp_human_y < goal.y
                temp_human_y = temp_human_y + 1
            elseif temp_human_y > goal.y
                temp_human_y = temp_human_y - 1
            end
            return pedestrian_state(temp_human_x, temp_human_y, goal)
        end
        
        if rand_num <= 0.7
            # move human towards goal
            new_human = move_human_towards_goal(human, goal)
        elseif rand_num > 0.7 && rand_num <= 0.8
            new_human = move_human_towards_goal(human, human_fixed_goals[1])
        elseif rand_num > 0.8 && rand_num <= 0.9
            new_human = move_human_towards_goal(human, human_fixed_goals[2])
        elseif rand_num > 0.9
            new_human = move_human_towards_goal(human, human_fixed_goals[3])
        end
        return new_human
    end

    new_pedestrians = []
    
    # action 0
    if a == 0
        # kart state +2 steps based on path
        # x = new state in path's X
        # y = new state in path's Y
        # theta = new states - one previous state {if change in x or change in y}
        # v = v
        new_v = s.cart.v + a
        new_position = s.path[s.path_covered_index + new_v]
        new_theta = calculate_theta(new_position, s.path[s.path_covered_index + new_v - 1])
        cart_new_state = cart_state(new_position[1], new_position[2], new_theta, new_v)
        
        # pedestrians state +1 step in their path for all pedestrians
        # change x
        # change y
        for human in s.pedestrians
            new_human = update_human_state(human, s.pedestrian_goals, rng)
            push!(new_pedestrians, new_human)
        end
        # path {need to change now/later based on A* from kart's current position to goal}
        new_path_index = s.path_covered_index + new_v
    
    # action 1
    elseif a == 1
        # kart state +3 steps based on path
        # x = new state in path's X
        # y = new state in path's Y
        # theta = new states - one previous state {if change in x or change in y}
        # v = v +1
        new_v = s.cart.v + a % 5
        new_position = s.path[s.path_covered_index + new_v]
        new_theta = calculate_theta(new_position, s.path[s.path_covered_index + new_v - 1])
        cart_new_state = cart_state(new_position[1], new_position[2], new_theta, new_v)
            
        # pedestrians state +1 step in their path for all pedestrians
        # change x
        # change y
        for human in s.pedestrians
            new_human = update_human_state(human, s.pedestrian_goals, rng)
            push!(new_pedestrians, new_human)
        end
        
        # path {need to change now/later based on A* from kart's current position to goal}
        new_path_index = s.path_covered_index + new_v
        
    # action -1
    elseif a == -1
        # kart state +1 steps based on path
        # x = new state in path's X
        # y = new state in path's Y
        # theta = new states - one previous state {if change in x or change in y}
        # v = v -1
        new_v = s.cart.v + a
        if new_v < 0
            new_v = 0
        end
        new_position = s.path[s.path_covered_index + new_v]
        new_theta = calculate_theta(new_position, s.path[s.path_covered_index + new_v - 1])
        cart_new_state = cart_state(new_position[1], new_position[2], new_theta, new_v)
        
        # pedestrians state +1 step in their path for all pedestrians
        # change x
        # change y
        for human in s.pedestrians
            new_human = update_human_state(human, s.pedestrian_goals, rng)
            push!(new_pedestrians, new_human)
        end
        
        # path {need to change now/later based on A* from kart's current position to goal}
        new_path_index = s.path_covered_index + new_v
        
    end
    
    # update the state object
    sp = SP_POMDP_state(cart_new_state, new_pedestrians, s.pedestrian_goals, s.path, new_path_index)
#     sp.cart = cart_new_state
#     sp.pedestrians = new_pedestrians
#     sp.pedestrian_goals = s.pedestrian_goals
#     sp.path = s.path
#     sp.path_covered_index = new_path_index

    
    # observation model
    o = new_pedestrians
    
    # reward model
    
    # collision reward
    function collision_reward(sp, coll_threshold)
        total_reward = 0
        cart_pose_x = sp.cart.x
        cart_pose_y = sp.cart.y
        for human in sp.pedestrians
            dist = ((human.x - cart_pose_x)^2 + (human.y - cart_pose_y)^2)^0.5
            if dist < coll_threshold
                total_reward = total_reward - 10
            end
        end
        return total_reward
    end
    
    # goal reward
    function goal_reward(sp, s, goal_threshold)
        total_reward = -1
        cart_new_pose_x = sp.cart.x
        cart_new_pose_y = sp.cart.y
        
        cart_goal = sp.path[length(sp.path)]
        new_dist = ((cart_goal[1] - cart_new_pose_x)^2 + (cart_goal[1] - cart_new_pose_y)^2)^0.5
        
        cart_old_pose_x = sp.cart.x
        cart_old_pose_y = sp.cart.y
        old_dist = ((cart_goal[1] - cart_old_pose_x)^2 + (cart_goal[1] - cart_old_pose_y)^2)^0.5
        
        if new_dist < old_dist && new_dist != 0
            total_reward = 100/new_dist
        elseif new_dist == 0
            total_reward = 100
        end
        return total_reward
    end
    
    # speed reward
    function speed_reward(sp, max_speed)
        return (sp.cart.v - max_speeed)/max_speed
    end
    
    r = collision_reward(sp, 2) + goal_reward(sp, s, 0) + speed_reward(sp, 5)
    
    
    # create and return a NamedTuple
    return (sp=sp, o=o, r=r)
    
end

In [ ]:
g1 =  human_goal_location(4,5)
ps1 = pedestrian_state(1,1,g1)
ps2 = pedestrian_state(2,2,g1)
cs = cart_state(1,2,3,4)
pd = SP_POMDP_state(cs,[],[])
push!(pd.pedestrians,ps1)
push!(pd.pedestrians,ps2)
pd.pedestrians[2].goal

In [ ]:
@show(pd.pedestrians[1].goal)

In [ ]:
t = -1
if t != 0
    @show(t)
end
    

In [ ]:
function calculate_theta(current_state, previous_state)
    theta = 0
    x_diff = current_state[1] - previous_state[1]
    y_diff = current_state[2] - previous_state[2]
    if x_diff != 0
        if x_diff < 0
            theta = 90
        else
            theta = 270
        end
    end
    if y_diff != 0
        if y_diff < 0
            theta = 0
        else
            theta = 180
        end
    end
    return theta
end

In [ ]:
theta = calculate_theta((2,4), (3,4))


In [ ]:
ped = []

push!(ped, 1)
push!(ped, 2)

@show(ped)

In [ ]:
using Random

rng = MersenneTwister()
rand_num = rand(rng)

In [ ]:
@show(rand_num)

In [ ]:
a = []
push!(a, (1,1))
push!(a, (2,2))
push!(a, (3,3))

@show(a)

idx = findall(x -> x==(2,2), a)

@show(idx)
    
deleteat!(a, findall(x -> x==(2,2), a)[1]) # => ["D", "A", "t"]

In [ ]:
function test_pass_by_ref(human)
    new_human = pedestrian_state(10, 20, human.goal)
#     new_human.x = 10
#     new_human.y = 20
    return new_human
end


g1 =  human_goal_location(4,5)
ps1 = pedestrian_state(1,1,g1)
@show(ps1)

new_h = test_pass_by_ref(ps1)
@show(ps1)
@show(new_h)

In [ ]:
a = [1,2,3]

function test_array(a)
    b = a
    @show(b)
    b[1] = 11
    @show(a)
    @show(b)
end

test_array(a)
@show(a)

In [ ]:
# sppp = SP_POMDP_state()

In [ ]:
function upper_bound(m, b)
    value_sum = 0.0
    for (s, w) in weighted_particles(b)
        if iscollision(s)
            value_sum += w*m.r_collision
        else
            value_sum += discount(m)^time_to_goal(s)*m.r_goal
        end
    end
    return value_sum/weight_sum(b)
end

In [ ]:
struct human_goal_location
    x:: Int64
    y:: Int64
end

struct pedestrian_state
    x:: Int64
    y:: Int64
    goal:: human_goal_location
end

struct cart_state
    x:: Int64
    y:: Int64
    theta:: Int64
    v:: Int64
end    

struct observations
    observed_human_positions:: Array{human_goal_location}
end

struct SP_POMDP_state
    cart:: cart_state
    pedestrians:: Array{pedestrian_state}
    pedestrian_goals:: Array{human_goal_location}
    path_covered_index:: Int64
end

struct human_goal_probability
    distribution::Array{Float64}
end

function isgoalstate(s)
    cart_x = s.cart.x
    cart_y = s.cart.y
    cart_goal = m.cart_goal_position
    if(cart_goal.x == cart_x && cart_goal.y == cart_y)
        return true
    else
        return false
    end
end

In [ ]:
mutable struct Speed_Planner_POMDP <: POMDPs.POMDP{SP_POMDP_state,Int,observations}
    discount_factor::Float64
    step_size::Int
    collision_threshold::Float64
    goal_reward::Int64
    max_cart_speed::Int64
    cart_goal_position::human_goal_location
    starting_cart_state::cart_state
    starting_human_states::Array{pedestrian_state}
    fixed_human_goal_locations::Array{human_goal_location}
    human_goals_prob_distribution::Array{human_goal_probability}
    astar_path::Array{Tuple{Int64,Int64},1}
    start_path_index::Int64
end
Speed_Planner_POMDP() = Speed_Planner_POMDP(0.9,1,2,100,5,)
discount(p::Speed_Planner_POMDP) = p.discount_factor
isterminal(::Speed_Planner_POMDP, s::SP_POMDP_state) = isgoalstate(s);

In [ ]:
d1 = human_goal_probability(0.15,0.3,0.5,0.05)
d2 = human_goal_probability(0.1,0.3,0.1,0.5)
d3 = human_goal_probability(0.5,0.25,0.1,0.15)
d4 = human_goal_probability(0.05,0.5,0.35,0.1)

In [ ]:
lk = [d1,d2,d3,d4]

In [ ]:
typeof(lk)

In [ ]:
lk[1].prob_g1

In [ ]:
q = [0.1,0.3,0.5,0.1]

In [ ]:
push!(q,0.77)

In [24]:
robot_path = [( 8, 30),( 7, 30),( 7, 29),( 7, 28),( 7, 27),( 7, 26),( 7, 25),
    ( 7, 24),( 7, 23),( 7, 22),( 6, 22),( 6, 21),( 5, 21),( 5, 20),( 5, 19),
             ( 5, 18),( 5, 17),( 5, 16),( 5, 15),( 5, 14),( 5, 13),( 5, 12),( 5, 11),
             ( 5, 10),( 5,  9),( 5,  8),( 5,  7),( 5,  6),
             ( 5,  5),( 5,  4),( 5,  3),( 5 , 2),( 5 , 1),( 6,  1),( 7,  1)]

35-element Array{Tuple{Int64,Int64},1}:
 (8, 30)
 (7, 30)
 (7, 29)
 (7, 28)
 (7, 27)
 (7, 26)
 (7, 25)
 (7, 24)
 (7, 23)
 (7, 22)
 (6, 22)
 (6, 21)
 (5, 21)
 ⋮      
 (5, 10)
 (5, 9) 
 (5, 8) 
 (5, 7) 
 (5, 6) 
 (5, 5) 
 (5, 4) 
 (5, 3) 
 (5, 2) 
 (5, 1) 
 (6, 1) 
 (7, 1) 

In [25]:
typeof(robot_path)

Array{Tuple{Int64,Int64},1}

In [ ]:
robot_path[1][1]

In [ ]:
g1 =  goal_location(1,1)
g2 =  goal_location(1,30)
g3 =  goal_location(14,30)
g4 =  goal_location(14,1)

cart_start_state = cart_state(8,30,0,2)
cart_goal = goal_location(7,1)

ps1 = pedestrian_state(3,22,g4)
ps2 = pedestrian_state(4,3,g3)
ps1 = pedestrian_state(11, 22,g2)
ps2 = pedestrian_state(12,4,g1)

human_state_start_list = [ps1,ps2,ps3,ps4]

cs = cart_state(1,2,3,4)
pd = SP_POMDP_state(cs,[],[],1)
push!(pd.pedestrians,ps1)
push!(pd.pedestrians,ps2)
pd.pedestrians[2].goal



In [15]:
mutable struct LightDark1D <: POMDPs.POMDP{Float64,Int,Int}
    discount_factor::Float64
    correct_r::Float64
    incorrect_r::Float64
    step_size::Int
    movement_cost::Float64
end
ld1() = LightDark1D(0.9, 10, -10, 1, 0)
discount(p::LightDark1D) = p.discount_factor
isterminal(::LightDark1D, s::Float64) = isnan(s);

In [17]:

actions(::LightDark1D) = [-1, 0, 1] # Left Stop Right

initialstate_distribution(pomdp::LightDark1D) = Normal(2.0,3.0)

initialstate_distribution (generic function with 11 methods)

In [23]:
typeof(ld1())

LightDark1D

In [19]:
actions(ld1())

3-element Array{Int64,1}:
 -1
  0
  1